# Spark

Hoewel het MapReduce algoritme van Hadoop een aantal voordelen heeft. 
De meest beperkende eigenschap van het MapReduce algoritme is de snelheid.
Omdat alles ingelezen wordt vanaf de harde schijf, tussenresultaten op de schijf opgeslagen worden en de finale resultaten ook wordt er tot wel 90% van de rekentijd gespendeerd in lees- of schrijfopdrachten.

Spark is geintroduceerd om dit te versnellen door gebruik te maken van in-memory processing.
Hierdoor is Spark tot 3 keer sneller op grote datasets en tot 100 keer op kleinere datasets.

Het spark framework kan gebruik maken van een externe opslag-locatie voor bestanden bij te houden (zoals HDFS) en bestaat uit de volgende componenten:
* SparkCore
* Spark SQL
* Spark Streaming
* MLlib
* SparkGraph

Daarnaast zijn er ook verschillende Spark Api's voor verschillende programmeertalen zoals Python, Scala, Java, ...
Hierdoor is het framework ook flexibeler dan het standaard MapReduce algoritme.
Heel veel informatie over het spark framework vind je in de [documentatie](https://spark.apache.org/docs/latest/quick-start.html) en de programming guides (bovenaan).

In [1]:
import pydoop.hdfs as hdfs

In [2]:
localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/Spark'):
    client.create_directory('/user/bigdata/Spark')
client.set_working_directory('/user/bigdata/Spark')
print(client.working_directory())

# do some cleaning in case anything else than input is present on HDFS
for f in client.list_directory("."):
    if not f["name"].endswith("input.txt"):
        client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("input.txt", client, "input.txt")

2023-03-09 14:22:10,763 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/user/bigdata/Spark


0

## Installatie

Een python implementatie van Spark kan eenvoudig geinstalleerd worden door het volgende commando uit te voeren. 
Dit moet maar eenmalig gebeuren.
Om te kijken of het reeds geinstalleerd is kan je kijken naar de versie van pyspark (indien geinstalleerd). 
Als de versie correct gereturned wordt, dan is het reeds geinstalleerd.

In [ ]:
!pip install pyspark

In [3]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.17
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


Spark kan op drie manieren werken:
* Boven op MapReduce (traag)
* Boven op Yarn
* Via zijn eigen resource manager

In deze notebook gaan we gebruik maken van Spark gebruikmakende van yarn.   

## Resilient Distributed Datasets



In [5]:
# SparkContext: geeft aan hoe de cluster/storage bereikt kan worden
# conf: configuration van de applicatie
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

Voor de configuratie moeten we vooral twee zaken aangeven, namelijk:
* Naam van de applicatie (is zichtbaar in de yarn)
* Master url. De url dat het type cluster en hoe het te bereiken aangeeft. Wij gaan vooral werken met local om te communiceren met het lokale bestandssysteem 

In [6]:
conf = SparkConf().setAppName('test').setMaster('yarn') # dit is gewoon configuratie
sc = SparkContext(conf=conf)  # deze sparkcontext kan gebruikt worden om rdd's aan te maken of files in te lezen

spark = SparkSession.builder.getOrCreate()
# deze maakt intern een sparksessie aan als deze nog niet bestaat, anders gebruikt hij de reeds bestane
# dit is belangrijk omdat er maar 1 sparksessie tegelijkertijd actief kan zijn
# heeft intern een sparkcontext: spark.sparkContext dus de eerste twee regels zijn strikt gezien niet meer nodig

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-03-09 14:49:25,309 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-03-09 14:49:39,281 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## Wordcount voorbeeld

Om de api van pyspark te leren kennen kan je gaan naar de [documentatie](https://spark.apache.org/docs/latest/api/python/reference/index.html).
Een eerder stap bij stap uitleg kan je [hier](https://spark.apache.org/docs/latest/api/python/getting_started/index.html) vinden.

In onderstaande code gaan we stap voor stap het wordcount-voorbeeld uitwerken.

Eerst moet er een pyspark context aangemaakt worden als volgt.

In [7]:
textFile = spark.read.text('Spark/input.txt')
textFile.collect() 
# deze gaat de hele dataset van de cluster lokaal brengen
# pas hier dus mee op in het geval van grote datasets
# je maakt ook niet meer gebruik van de rekenrkacht van de cluster als je met dit resultaat verder werkt

[Row(value='Hello World,'),
 Row(value='hello world,'),
 Row(value='hello world,'),
 Row(value=''),
 Row(value='Dit is een voorbeeld file om het Wordcount voorbeeld te testen !')]

In [11]:
print(textFile.count())
print(textFile.first())
print(textFile.filter(textFile.value.contains('Hello')).collect())

5
Row(value='Hello World,')
[Row(value='Hello World,')]



[Stage 16:>                                                         (0 + 1) / 1]



In [18]:
# splits de lijn in woorden, maak van elk woord een aparte lijn
words = sc.textFile('Spark/input.txt').flatMap(lambda line: line.split(' ')) 
result = words.map(lambda w: (w, 1)).reduceByKey(lambda a, b: a+b)

#reduce by key
#woord 1 -> [1,4,3,2,5,1,2,5] -> (a=1, b=4) -> 1 (a)+4 (b)=5 (nieuwe a) -> (nieuwe a + b=3) = 8 (nieuwe a) -> ...

result.saveAsTextFile('Spark/output.txt')

**Wat gebeurt er in dit voorbeeld?**

Sparkcontext om een connectie te maken met de distributed storage
De input file wordt dan ingelezen met de textFile functie.
Door middel van de flatMap functie wordt de tekst lijn per lijn ingelezen en gesplits in woorden. 
Dit resulteert in een RDD (Resilient Distributed Dataset.
De .map() functie maakt een key-value pair aan voor elke keer dat het woord voorkomt.
In een laatste fase is er een reduce stap per key die de som neemt van alle keren dat het woord voorkomt om de uiteindelijke wordcount te nemen.
Of af te ronden wordt het resultaat opgeslagen.

![spark wordcount in yarn](images/yarn_001.png)

## SparkSession

Nu gaan we stuk voor stuk de verschillende stappen bekijken om een pyspark applicatie te maken.
De eerste stap is het aanmaken van een sessie (SparkSession) wat het beginpunt is voor spark applications.
Er zijn twee manieren om een SparkSession aan te maken:
* builder()
* newSession()

Bij het aanmaken van een session wordt er intern een SparkContext object aangemaakt. 
Dit object stelt de connectie naar een cluster voor.
Er kan maar 1 context tegelijkertijd actief zijn.
Als je wil connecteren met een tweede cluster moet je eerst stop() oproepen op de reeds actieve context.

In [19]:
# als je een andere sparksessie wil moet je het vorige eerst stopzetten
spark.stop()

# met master kan je spark lokaal draaien (tussen rechte haakjes staat het aantal cpu cores)
# heel wat andere opties zijn mogelijk, bvb als er extra zaken moeten toegevoegd worden voor connectie met een database
spark = SparkSession.builder.master('local[1]').appName('lesSpark').getOrCreate()

spark.stop()

spark = SparkSession.builder.config('spark.driver.cores', 2).appName('lesSpark').getOrCreate()


## RDD

Op basis van het SparkSession object is het dan mogelijk om RDD-objecten aan te maken.
Een RDD is de basis dataobject binnen Spark dat in parallel op verschillende nodes binnen een cluster kan uitgevoerd worden.
Alle dataobjecten binnen spark horen tot deze klassen en dus zijn er veel mogelijkheden om RDD's aan te maken.
Hier haal ik er twee aan:
* parallelize() om bestaande python objecten om te zetten naar een RDD
* textFile() of andere read methoden om bestanden op de cluster uit te lezen

In [21]:
# aanmaken van rdds
# meest manuele manier -> maak eerst een lokaal object (lijstje van tuples) -> parallelize -> dan staat het op de cluster in ram
dataList = [('jens', 8), ('harry', 6), ('ron', 4)]
rdd = spark.sparkContext.parallelize(dataList)
rdd.collect()

# andere manier -> om data in te lezen
rdd2 = spark.read.text('Spark/input.txt') # dit geeft een DataFrame terug ipv van een rdd -> gemakkelijker te manipuleren
rdd3 = spark.sparkContext.textFile('Spark/input.txt')
print(rdd2)
print(rdd3)

DataFrame[value: string]
Spark/input.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0


Met bovenstaande methoden hebben we twee rdd's aangemaakt. 
Op deze objecten kunnen nu verscheidene operaties uitgevoerd worden.
Een belangrijke eigenschap van dit type objecten is dat ze steeds in parallel uitgevoerd worden.

De beschikbare operaties kunnen in twee groepen verdeeld worden:
* transformaties
* acties

[Transformaties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-transformations/) zijn lazy-operations waarvoor de berekening uitgesteld wordt en geven een nieuw RDD terug.
Een aantal voorbeelden van transformaties zijn:
* flatMap()
* map()
* reduceByKey()
* filter()
* sortByKey()

[Acties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-actions/) zijn operaties die een berekening starten (ook van de nodige transformaties) en geven een niet RDD-object terug. 
Een aantal voorbeelden hiervan zijn:
* count()
* collect()
* first()
* max()
* reduce()

Lees nu bovenstaande links en geef de functies die nodig zijn voor de volgende vragen op te lossen. Geef ook aan of het transformaties zijn of acties:
* Het aantal keer dat elke waarde aanwezig is in de dataset (1 functie voor wordcount uit te voeren)
* Uitfilteren van rijen
* Groeperen van een aantal rijen op basis van een bepaalde waarde.
* Toevoegen van een kolom aan elke key (bvb de lengte van een woord)
* Hoe doe je head() uit pandas op RDD's?
* Hoe doe je de apply() uit pandas op RDD's?

Maak nu een spark applicatie dat van de eerste RDD (met de studenten) telt hoeveel studenten geslaagd zijn.

In [24]:
#dataList = [('jens', 8), ('harry', 6), ('ron', 4)]
rdd.filter(lambda student: student[1] >= 5).count()

2

De applicatie voor het berekenen van een gemiddelde is iets complexer.
Dit soort applicaties kan geschreven worden als volgt:

In [29]:
# omdat we niet werken met dataframes zitten we met beperkte functionaliteit
# best in twee stappen doen met rdds
aantal = rdd.count()
rdd.map(lambda x: x[1]).reduce(lambda x, y: (x+y)) /aantal
# let op: aantal niet binnen de lambda functie plaatsen -> anders bekom je onderstaande berekening
#((8+6) / aantal + 4) / aantal

6.0

Schrijf nu een mapreduce applicatie in spark om de tweede RDD van de input te verwerken en het aantal woorden van elke lengte te bekomen.
Tussenresultaten k

In [31]:
words = rdd3.flatMap(lambda line: line.split(' ')) 
result = words.map(lambda w: (len(w), 1)).reduceByKey(lambda a, b: a+b)

result.collect()

[(5, 3), (6, 4), (0, 1), (3, 3), (2, 3), (9, 3), (4, 1), (1, 1)]

## Dataframes

Een belangrijke subklasse van RDD's zijn dataframes.
Dit is een veel gebruikte manier om gestructureerde data voor te stellen.
Dataframes in spark is sterk gerelateerd aan de dataframes gezien in pandas.
Het belangrijskte verschil is dat ze verdeeld worden over de cluster en operaties op de dataframes in parallel uitgevoerd worden.
Dataframes kunnen aangemaakt worden door gebruik te maken van de createDataFrame functie in context of ingelezen worden vanuit csv's of jsons. Ten slotte kunnen dataframes ook komen van externe bronnen zoals databases als resultaat van een sql-query.

In [42]:
data = [('HarryHarryHarryHarryHarryHarry', 'Potter','1980-07-31','M',100000000),
  ('Ronald','Wemel','1980-04-01','M',10),
  ('Hermelijn','Griffel','1979-09-19','F',4000)
]

columns = ["firstname","lastname","dob","gender","budget"]

import pandas as pd
display(pd.DataFrame(data, columns=columns))

df = spark.createDataFrame(data=data, schema=columns)
#df.show(2)
df.show(truncate=False) # print tekst kolommen volledig uit
df.show(vertical=True) # print kolommen verticaal/onder elkaar uit. is leesbaarder als er veel kolommen zijn

df.describe().show()

,firstname,lastname,dob,gender,budget
0,HarryHarryHarryHarryHarryHarry,Potter,1980-07-31,M,100000000
1,Ronald,Wemel,1980-04-01,M,10
2,Hermelijn,Griffel,1979-09-19,F,4000


+------------------------------+--------+----------+------+---------+
|firstname                     |lastname|dob       |gender|budget   |
+------------------------------+--------+----------+------+---------+
|HarryHarryHarryHarryHarryHarry|Potter  |1980-07-31|M     |100000000|
|Ronald                        |Wemel   |1980-04-01|M     |10       |
|Hermelijn                     |Griffel |1979-09-19|F     |4000     |
+------------------------------+--------+----------+------+---------+

-RECORD 0-------------------------
 firstname | HarryHarryHarryHa... 
 lastname  | Potter               
 dob       | 1980-07-31           
 gender    | M                    
 budget    | 100000000            
-RECORD 1-------------------------
 firstname | Ronald               
 lastname  | Wemel                
 dob       | 1980-04-01           
 gender    | M                    
 budget    | 10                   
-RECORD 2-------------------------
 firstname | Hermelijn            
 lastname  | Griffe

+-------+--------------------+--------+----------+------+-------------------+
|summary|           firstname|lastname|       dob|gender|             budget|
+-------+--------------------+--------+----------+------+-------------------+
|  count|                   3|       3|         3|     3|                  3|
|   mean|                null|    null|      null|  null|         3.333467E7|
| stddev|                null|    null|      null|  null|5.773386936614157E7|
|    min|HarryHarryHarryHa...| Griffel|1979-09-19|     F|                 10|
|    max|              Ronald|   Wemel|1980-07-31|     M|          100000000|
+-------+--------------------+--------+----------+------+-------------------+



## PySpark SQL

Bovenstaande datastructuren (RDD's en Dataframes) zijn een onderdeel van het Pyspark sql module.
De Spark API heeft een hele reeks methoden en functies om deze in te laden, uit te lezen en te manipuleren.
Daarnaast maakt deze module het ook mogelijk om SQL-queries uit te voeren op dataframes.
Om SQL-queries uit te voeren op dataframes moet er eerst een view gemaakt worden in het dataframe met de functie createOrReplaceTempView("view_name")

Daarna kan je gebruik maken van de .sql() functie om allerhande sql queries uit te voeren.

In [48]:
# je kan sql queries uitvoeren op dataframes
# stap 1: zeg dat dit dataframe een tabel is met de naam ...
df.createOrReplaceTempView('harry')
# stap 2: voer queries
spark.sql('select * from harry where gender="M"').show()
df.filter(df.gender == 'M').show()


spark.sql('select gender, count(*) from harry group by gender').show()
df.groupby('gender').count().show()


df.select('firstname', 'lastname').show()

+--------------------+--------+----------+------+---------+
|           firstname|lastname|       dob|gender|   budget|
+--------------------+--------+----------+------+---------+
|HarryHarryHarryHa...|  Potter|1980-07-31|     M|100000000|
|              Ronald|   Wemel|1980-04-01|     M|       10|
+--------------------+--------+----------+------+---------+

+--------------------+--------+----------+------+---------+
|           firstname|lastname|       dob|gender|   budget|
+--------------------+--------+----------+------+---------+
|HarryHarryHarryHa...|  Potter|1980-07-31|     M|100000000|
|              Ronald|   Wemel|1980-04-01|     M|       10|
+--------------------+--------+----------+------+---------+

+------+--------+
|gender|count(1)|
+------+--------+
|     F|       1|
|     M|       2|
+------+--------+

+------+-----+
|gender|count|
+------+-----+
|     F|    1|
|     M|    2|
+------+-----+

+--------------------+--------+
|           firstname|lastname|
+-------------

Buiten de functionaliteit om SQL queries uit te voeren is ook het lezen en schrijven van allerhande dataformaten een belangrijk onderdeel van de pyspark sql module.
Meer informatie hierover kun je [hier](https://spark.apache.org/docs/latest/sql-data-sources.html) vinden in de documentatie.
In essentie ziet de code er uit als volgt:

In [ ]:
df = spark.read.option('delimiter', ';').option('header', True).csv('path_to_csv')

De opties die hierbij gekozen kunnen worden kun je vinden in de documentatie.

Daarnaast zijn er ook functionaliteiten om data uit te lezen speciaal voor Machine Learning zoals libsvm en image-directories maar die worden later getoond.

### Oefening

Net zoals RDD kunnen er een aantal operaties uitgevoerd worden op deze dataframes.
Een volledige lijst met alle operaties kan je [hier](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html) vinden.
Zoek de functies die gebruikt moeten worden om de volgende zaken uit te voeren:
* Groepeer volgens een bepaalde sleutel
* Krijg een lijst met alle kolomnamen
* Filter rijen uit
* Verwijder null-values in de dataset via rijen
* Verwijder null-values door kolommen te verwijderen
* Bereken een dataframe met statistieken van het dataframe
* Krijg een dataframe met alle nan waarden
* Hoe krijg je informatie zoals .info()
* Hoe werkt het groeperen van informatie op basis van een key/kolom

Probeer deze ook uit op bovenstaand aangemaakt dataframe

In [51]:
# groupby
# .columns
# filter
# dropna()
# drop (eerst checken )
# describe
# isnan() of isnull()
# .schema of printSchema
df.printSchema()
# groupby -> dit resulteert in iets dat geen dataframe
# -> er is nog een aggregatie nodig (group -> 1 rij) om een dataframe te bekomen
df.groupby('gender').count()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- budget: long (nullable = true)



DataFrame[gender: string, count: bigint]

Lees daarna volgende [link](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/) om een idee te krijgen over hoe verschillende functies uit te voeren op deze dataframes.
Werk nu de volgende oefening uit en maak hiervoor een spark applicatie:
* Download de iris dataset in sklearn
* Bewaar deze dataset als csv en upload de file naar het hdfs
* Schrijf de code om de csv uit te lezen en om te zetten naar een dataframe
* Print het dataschema uit voor het dataframe, hoeveel kolommen zijn er aanwezig in het dataframe.

**Extra vragen week 5**
* Bereken het minimum en maximum van de 'sepal width (cm)' en 'petal width (cm)' kolom.
* Hernoem de target kolom naar label
* Hernoem de labels 0 naar Soort 0, labels 1 naar Soort 2 en labels 3 naar Soort 3
* Voer normalisatie van de eerste 4 kolommen uit (het gemiddelde ervan aftrekken en delen door de standaardafwijking)
* Controleer de voorgaande stap door opnieuw het gemiddelde en de standaardafwijking te berekenen. Deze moeten respectievelijk 0 en 1 zijn.
* Bereken de oppervlakte van sepal door de lengte en breedte ervan te vermenigvuldigen. Noem deze nieuwe kolom sepal area. Doe dit ook voor de petal.
* Groepeer nu de rijen op de label kolom. Bereken per groep het gemiddelde van elke kolom. Is er een verschil tussen de verschillende klassen?

In [52]:
from sklearn.datasets import load_iris
import pandas as pd

df = load_iris(as_frame=True).frame
df.to_csv("input.csv")
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [54]:
if not client.exists('/user/bigdata/Spark/demo'):
    client.create_directory('/user/bigdata/Spark/demo')
client.set_working_directory('/user/bigdata/Spark/demo')

# do some cleaning in case anything else than input is present on HDFS
for f in client.list_directory("."):
    if not f["name"].endswith("input.csv"):
        client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("input.csv", client, "input.csv")

0

In [110]:
# nuttige functies
from pyspark.sql.functions import col, avg, sum, count
import pyspark.sql.functions as f

df = spark.read.option('delimited', ',', ).option('header', True).csv('Spark/demo/input.csv')
#df.printSchema()

# drop de id kolom
df = df.drop('_c0')
#df.show(2)

# 1 colom selecteren
#df.select('sepal width (cm)').show(1)
# gemiddelde berekenen
#df.select(avg('sepal width (cm)')).show(1) # met de eerste import
#df.select(f.avg('sepal width (cm)')).show(1) # met de tweede import
# andere naam geven
#df.select(avg('sepal width (cm)').alias('avg sw')).show(1) 
# meerdere kolommen tegelijkertijd
df_avg = df.select([f.avg(c).alias('avg ' + c) for c in df.columns]).collect()[0]
print(df_avg)

# filteroperaties
#df.select([col(c) > 3 for c in df.columns]).show(2)
#df.select([col(c).isNull() for c in df.columns]).show(2)
# welke getallen groter dan het gemiddelde
df_avg_greater_than_avg = df.select([(col(c) > df_avg[index]).alias(c) for index,c in enumerate(df.columns)])
# hoeveel zijn er zo per kolom
df_avg_greater_than_avg.printSchema()
# alias was belangrijk, anders vond spark de kolommen niet
df_avg_greater_than_avg.select([f.sum(col(c).cast('int')) for c in df_avg_greater_than_avg.columns]).show(1)
df_avg_greater_than_avg.select([f.count(f.when(col(c), 1)) for c in df_avg_greater_than_avg.columns]).show(1)

# eerst alles omzetten naar een integer
tmp = df_avg_greater_than_avg.select([(col(c).cast("int")).alias(c) for c in df_avg_greater_than_avg.columns])
c = df_avg_greater_than_avg.columns
# kolom toevoegen -> som van de 4 features
df2 = tmp.withColumn("aantal", col(c[0]) + col(c[1]) + col(c[2]) + col(c[3]) + col(c[4]))
df2.show(3)

# opbouwen van een if-else constructie om een kolom aan te maken
df3 = df.withColumn("class", f.when(col("target") == 0, "klasse 0")
                     .when(col("target") ==1, "klasse 1")
                     .otherwise("klasse 2"))
df3.show(5)
df3.tail(5)

df3.filter(col("target") ==2).show()

Row(avg sepal length (cm)=5.843333333333335, avg sepal width (cm)=3.057333333333334, avg petal length (cm)=3.7580000000000027, avg petal width (cm)=1.199333333333334, avg target=1.0)
root
 |-- sepal length (cm): boolean (nullable = true)
 |-- sepal width (cm): boolean (nullable = true)
 |-- petal length (cm): boolean (nullable = true)
 |-- petal width (cm): boolean (nullable = true)
 |-- target: boolean (nullable = true)

+-----------------------------------+----------------------------------+-----------------------------------+----------------------------------+------------------------+
|sum(CAST(sepal length (cm) AS INT))|sum(CAST(sepal width (cm) AS INT))|sum(CAST(petal length (cm) AS INT))|sum(CAST(petal width (cm) AS INT))|sum(CAST(target AS INT))|
+-----------------------------------+----------------------------------+-----------------------------------+----------------------------------+------------------------+
|                                 70|                              

**Shared variabelen**

Variabelen met read-write acces zijn zeer inefficient om te gebruiken in een cluster met sterke parallelisatie.
Spark bied echter twee varianten aan die wel efficient geimplementeerd kunnen worden, namelijk
* Broadcasted variabelen
* Accumulators

Broadcasted variabelen zijn read-only variabelen, die aangemaakt worden door de driver en eenmalig verspreid worden over de nodes in plaats van voor elke job.
Dit wordt vooral gebruikt om grote data die veelvuldig gebruikt wordt te cachen op de nodes.
Bij het gebruik van broadcast variabelen is het belangrijk om te onthouden dat je de originele variabele niet meer mag gebruiken na het aanmaken van de broadcasted variabele omdat ze anders toch nog elke job doorgestuurd wordt.
De belangrijkste functies om te werken met broadcasted variabelen zijn:

**Accumulators**

Het andere type dat aangeboden wordt zijn accumulators.
Deze laten enkel toe dat noden iets toevoegen aan een gedeelde variabele.
Enkel de driver kan deze variabele uitlezen.
Dit kan bijvoorbeeld gebruikt worden om tellers of sommen bij te houden.
Deze accumulators kunnen een naam hebben (named accumulators zijn zichtbaar in de wep api).
De ingebouwde accumulator van Spark ondersteunt enkel numerieke accumulators.
Het is echter mogelijk om eigen accumulators toe te voegen door over te erven van de AccumulatorParam klasse en deze twee functies te implementeren:
* zero: De begin waarde van de accumulator
* addInPlace: Om twee waarden samen te voegen

## Pandas

Door de hoge populariteit van pandas in python is er een alternatief uitgewerkt binnen de laatste versie van Spark (eind 2021) dat de pandas api integreert.
Hierdoor kan je code schrijven die identiek is aan te werken met pandas.
Lees [dit artikel](https://towardsdatascience.com/run-pandas-as-fast-as-spark-f5eefe780c45) om meer informatie te krijgen over de verschillen tussen de dataframes API en de pandas-on-spark API.
De documentatie voor deze api vind je [hier](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html).
Een belangrijk onderdeel van deze documentatie omvat de best practices:
* Check execution plan (dmv de explain() functie)
* Use checkpoints voor fout-tolerantie en efficientie van de planner.
 * df.spark.local_checkpoint()
* Vermeid data shuffling (sorting) omdat hierbij data tussen nodes moet gestuurd worden wat niet efficient is.
* Vermeid berekeningen op 1 partitie (geen parallellisatie)
* Vermeid kolomnamen startend of eindigend op "_"
 * Deze worden gebruikt door interne functies van pandas/spark
* Kolomnamen moeten uniek zijn
* Specificeer de index kolom bij omzetten dataframe en pandas-on-spark API
* Gebruik zoveel mogelijk van de pandas-on-spark API direct ipv standaard python functies om conflicten te vermijden
 * df.sum() werkt maar sum(df) niet
* Vermeid operaties op meerdere dataframes want deze gebruiken een join intern en is hierdoor traag